In [4]:
!pwd
%cd ../yolov5
!pwd

/home/sidd/Desktop/cricket_analysis/notebooks
/home/sidd/Desktop/cricket_analysis/yolov5
/home/sidd/Desktop/cricket_analysis/yolov5


In [5]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
from PIL import Image
Image(filename='runs/train/exp2/results.png', width=1000)  # view results.png

NameError: name 'Image' is not defined

In [ ]:
%%time
# multi GPU training
# train yolov5s on custom data for 100 epochs
# time its performance
# %cd ../yolov5/
!python train.py --img 640 --batch 16 --epochs 5 --data /home/sidd/Desktop/cricket_game_analysis/data/raw/cricket_dataset/data.yaml --weights 'yolov5x.pt' --device 0,1

In [ ]:
!pip install optuna --quiet

In [ ]:
import optuna
import subprocess

def objective(trial):
    img_size = trial.suggest_int('img_size', 320, 1024, step=32)
    batch_size = trial.suggest_int('batch_size', 8, 64, step=8)
    epochs = trial.suggest_int('epochs', 10, 30)
    device = trial.suggest_categorical('device', ['0,1'])

    data = '/home/sidd/Desktop/cricket_game_analysis/data/raw/cricket_dataset/data.yaml'
    weights = 'yolov5x.pt'

    command = f"python train.py --img {img_size} --batch {batch_size} --epochs {epochs} --data {data} --weights {weights} --device {device}"
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)

    print(f"\n[INFO] Hyperparameters: img_size={img_size}, batch_size={batch_size}, epochs={epochs}, device={device}\n")

    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode())

    rc = process.poll()
    print(f"[INFO] Return code: {rc}\n")

    return -rc

if __name__ == '__main__':
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=5)

    print(f"\n[INFO] Best hyperparameters: {study.best_params}")
    print(f"[INFO] Best return code: {study.best_value}")
    print(f"[INFO] Best trial: {study.best_trial.number}")
